# Automated ML
TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import os
import json
import requests
import joblib
import itertools

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix

from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset

from azureml.train.automl import AutoMLConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails

from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

## Dataset

### Overview
Dataset from Davide Chicco, Giuseppe Jurman: Machine learning can predict survival of patients with heart failure from serum creatinine and ejection fraction alone. BMC Medical Informatics and Decision Making 20, 16 (2020)

Cardiovascular diseases (CVDs) are the number 1 cause of death globally, taking an estimated 17.9 million lives each year, which accounts for 31% of all deaths worlwide.
Heart failure is a common event caused by CVDs and this dataset contains 12 features that can be used to predict mortality by heart failure:
age: age of the patient (years)
anaemia: decrease of red blood cells or hemoglobin (boolean)
high blood pressure: if the patient has hypertension (boolean)
creatinine phosphokinase (CPK): level of the CPK enzyme in the blood (mcg/L)
diabetes: if the patient has diabetes (boolean)
ejection fraction: percentage of blood leaving the heart at each contraction (percentage)
platelets: platelets in the blood (kiloplatelets/mL)
sex: woman or man (binary)
serum creatinine: level of serum creatinine in the blood (mg/dL)
serum sodium: level of serum sodium in the blood (mEq/L)
smoking: if the patient smokes or not (boolean)
time: follow-up period (days)
In this project, Azure AutoML will be used to Heart Failure Prediction with the use of 12 clinical features for predicting death events.

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl-experiment'

experiment=Experiment(ws, experiment_name)

In [ ]:
ds = dataset = Dataset.get_by_name(ws, 'heart-failure-clinical-records')

ds.take(5).to_pandas_dataframe()

In [ ]:
data_train, data_test = dataset.random_split(0.8)

## Create or Attach an AmlCompute cluster
Create a compute target for the AutoML run

In [ ]:
# Choose a name for your CPU cluster
aml_compute_cluster_name = "auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=aml_compute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, aml_compute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

## AutoML Configuration

The main goal of classification models is to predict which categories new data will fall into based on learnings from its training data.
AutoML typically performs cross validation, data balance check, cardinality check in prior to machine learning process with a variety of algorithms.

In [ ]:
# TODO: Put your automl settings here
automl_settings = {"experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'accuracy',
    "n_cross_validations": 4}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=data_train,
                             label_column_name="DEATH_EVENT",   
                             enable_early_stopping= True,
                             featurization= 'auto',
                             **automl_settings)

In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output = True)

## Run Details
`RunDetails` widget to show the different experiments.

In [ ]:
remote_run

In [ ]:
RunDetails(remote_run).show()

In [ ]:
remote_run.wait_for_completion()

## Best Model
TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model

In [ ]:
best_run, fitted_model = remote_run.get_output()
best_run_metrics = best_run.get_metrics()

In [ ]:
best_run

In [ ]:
fitted_model

In [ ]:
print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])
print(fitted_model._final_estimator)
print(best_run.get_tags())

In [ ]:
#TODO: Save the best model
os.makedirs('./outputs', exist_ok=True)

joblib.dump(fitted_model, filename='outputs/automl.joblib')

model_name = best_run.properties['model_name']
model_name

In [ ]:
env = best_run.get_environment()

script_file = 'score.py'

best_run.download_file('outputs/scoring_file_v_1_0_0.py', script_file)

## Model Deployment
Register the model, create an inference config and deploy the model as a web service

In [ ]:
model = remote_run.register_model(model_name = model_name,
                                  description = 'AutoML model')

In [ ]:
inference_config = InferenceConfig(entry_script = script_file, environment = env)

In [ ]:
aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

service_name = 'automl-heart-failure'

print(service_name)

Request to the web service to test it.

In [ ]:
%run endpoint.py

Print the logs of the web service and delete the service

In [ ]:
service = Model.deploy(ws, service_name, [model], inference_config, aci_config)
service.wait_for_deployment(True)
print("State: " + service.state)
print("Scoring URI: " + service.scoring_uri)

In [ ]:
service.delete()